In [ ]:

import pandas as pd
import torch
# @title Load Data


path='/content/DS-ALS-REDSEA-CLEANED.csv'  # @param {type: "string"}
df = pd.read_csv(path)
# @title PreProcessing Raw Data

Timestamp=df['Timestamp']
Speed=df['Speed']
Course=df['Course']
Latitude=df['Latitude']
Longitude=df['Longitude']
Vessel=df['Vessel']
new_df=pd.DataFrame({'Timestamp':Timestamp,'Speed':Speed,'Course':Course,'Latitude':Latitude,'Longitude':Longitude,'Vessel':Vessel})

#  REMOVE  VALUE masked     IN TABLE
df_cleaned = new_df.dropna()

df_cleaned = df_cleaned.replace('masked', pd.NA)
df_cleaned = df_cleaned.dropna()
df_cleaned['Latitude'] = df_cleaned['Latitude'].astype(float)
df_cleaned['Longitude'] = df_cleaned['Longitude'].astype(float)
df_cleaned['Speed'] = df_cleaned['Speed'].astype(float)
df_cleaned['Course']=df_cleaned['Course'].astype(float)

# Filter data based on latitude and longitude ranges
df_cleaned = df_cleaned[(df_cleaned['Longitude'] >= 32) & (df_cleaned['Longitude'] <= 44)]
df_cleaned = df_cleaned[(df_cleaned['Latitude'] >= 12) & (df_cleaned['Latitude'] <= 33)]

df_cleaned.sort_values(by=['Timestamp'], inplace=True)
df_cleaned.reset_index(drop=True, inplace=True)


vs=df_cleaned['Vessel'].unique()

# Group data by vessel and count the number of points in each trajectory
vessel_counts = df_cleaned.groupby('Vessel').size()

# Filter out vessels with more than 100 points
vessels_to_keep = vessel_counts[vessel_counts <= 150].index

# Filter the DataFrame to keep only the trajectories with 100 or fewer points
df_filtered = df_cleaned[df_cleaned['Vessel'].isin(vessels_to_keep)]

# Find min/max latitude and longitude for the filtered data
min_lat = df_filtered['Latitude'].min()
max_lat = df_filtered['Latitude'].max()
min_lon = df_filtered['Longitude'].min()
max_lon = df_filtered['Longitude'].max()
min_time=df_filtered['Timestamp'].min()
max_time=df_filtered['Timestamp'].max()
max_speed=df_filtered['Speed'].max()
min_speed=df_filtered['Speed'].min()
min_course=df_filtered['Course'].min()

max_course=df_filtered['Course'].max()


print(f"Min Latitude: {min_lat}")
print(f"Max Latitude: {max_lat}")
print(f"Min Longitude: {min_lon}")
print(f"Max Longitude: {max_lon}")
print(f"Min Timestamp: {min_time}")
print(f"Max Timestamp: {max_time}")
print(f"Min Speed: {min_speed}")
print(f"Max Speed: {max_speed}")
print(f"Min Course: {min_course}")
print(f"Max Course: {max_course}")
df_cleaned=df_filtered


In [ ]:
# @title Four-Hot

LAT_BINS = 100  # @param {type: "number"}
LON_BINS = 100  # @param {type: "number"}
SOG_BINS = 50  # @param {type: "number"}
COG_BINS = 10  # @param {type: "number"}

data=[]
labels=[]
features=['Latitude','Longitude','Speed','Course']
for v in vs:
  v1=df_cleaned[df_cleaned['Vessel']==v]
  fs=v1[features].values

  fs=torch.tensor(fs)
  fs[:,0]=fs[:,0]/LAT_BINS
  fs[:,1]=fs[:,1]/LON_BINS
  fs[:,2]=fs[:,2]/SOG_BINS
  fs[:,3]=fs[:,3]/360
  if len(fs)>2:
      data.append(fs)

Double_DATA=1   # @param {type: "number"}
data=data*Double_DATA


In [ ]:
# @title AISDataset

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import concurrent.futures

LAT, LON, SOG, COG, HEADING, TIMESTAMP = list(range(6))

class AISDataset(Dataset):
    def __init__(self, tracks, lat_bins, lon_bins, sog_bins, cog_bins,mean,divm=360):
        self.tracks = tracks  #
        self.lat_bins = lat_bins
        self.lon_bins = lon_bins
        self.sog_bins = sog_bins
        self.cog_bins = cog_bins
        self.total_bins = lat_bins + lon_bins + sog_bins + cog_bins  #

        # Normalize the mean values for inputs.
        self.mean = mean#self.calculate_mean()
        self.dataFourHot=[]
        self.dataTrackdFourHot=[]
        self.processdata(divm)


    def processdata(self,divm=360):
       outdata=self.get_four_hot_all_data(divm=divm)
       self.dataTrackdFourHot=outdata
       print('finsh get_four_hot_all_data',len(outdata))
       self.dataFourHot=[]
       for  i in range(len(outdata)):
         for j in range(len(outdata[i])):
           self.dataFourHot.append(outdata[i][j,:])

       print('finsh processdata')






    def get_four_hot_all_data(self, divm=360):
      results = []
      for i in range(len(self.tracks)):
              results.append(self.sparse_AIS_to_dense(self.tracks[i]))






      return results

    def sparse_AIS_to_dense_all(self,msgs_,divm=360):
        msgs_=msgs_/divm
        def create_dense_vect(msg, lat_bins=LAT_BINS, lon_bins=LON_BINS, sog_bins=SOG_BINS, cog_bins=COG_BINS):
            lat, lon, sog, cog = msg[0], msg[1], msg[2], msg[3]
            data_dim = lat_bins + lon_bins + sog_bins + cog_bins
            dense_vect = np.zeros(data_dim)
            dense_vect[int(lat * lat_bins)] = 1.0
            dense_vect[int(lon * lon_bins) + lat_bins] = 1.0
            dense_vect[int(sog * sog_bins) + lat_bins + lon_bins] = 1.0
            dense_vect[int(cog * cog_bins) + lat_bins + lon_bins + sog_bins] = 1.0
            return dense_vect

        #
        msgs_[msgs_ == 1] = 0.99999

        dense_msgs = []
        for msg in msgs_:
            dense_msgs.append(create_dense_vect(msg))

        dense_msgs = np.array(dense_msgs)
        return dense_msgs

    def sparse_AIS_to_dense_ones(self, msg):
        dense_vect = np.zeros(self.total_bins)
        dense_vect[int(msg[LAT] * self.lat_bins)] = 1.0
        dense_vect[int(msg[LON] * self.lon_bins) + self.lat_bins] = 1.0
        dense_vect[int(msg[SOG] * self.sog_bins) + self.lat_bins + self.lon_bins] = 1.0
        dense_vect[int(msg[COG] * self.cog_bins) + self.lat_bins + self.lon_bins + self.sog_bins] = 1.0
        return dense_vect
    def sparse_AIS_to_dense(self, msg_):
        dense_vect = np.zeros((len(msg_),self.total_bins))

        for i in range(len(msg_)):
          msg=msg_[i]

         # print(msg)
          dense_vect[i,np.int16(msg[0]*self.lat_bins)] = 1.0
          dense_vect[i,np.int16(msg[1]* self.lon_bins ) + self.lat_bins] = 1.0
          dense_vect[i,np.int16(msg[2]*self.sog_bins) + self.lat_bins + self.lon_bins] = 1.0
          dense_vect[i,np.int16(msg[3]* self.cog_bins) + self.lat_bins + self.lon_bins + self.sog_bins] = 1.0
        return dense_vect

    def dense_to_sparse(self, dense_msg, divm=360):
    # Initialize an empty list to store the sparse messages
      sparse_msgs = []

      # Iterate over the dense message (which represents four-hot encoded data for lat, lon, sog, cog)
      for msg in dense_msg:
          # Initialize variables to store lat, lon, sog, cog
          lat = -1
          lon = -1
          sog = -1
          cog = -1

          # Get lat (search in the first section of the vector)
          lat_bin = np.argmax(msg[:self.lat_bins])
          lat = lat_bin / self.lat_bins

          # Get lon (search in the second section of the vector)
          lon_bin = np.argmax(msg[self.lat_bins:self.lat_bins + self.lon_bins]) + self.lat_bins
          lon = (lon_bin - self.lat_bins) / self.lon_bins

          # Get sog (search in the third section of the vector)
          sog_bin = np.argmax(msg[self.lat_bins + self.lon_bins:self.lat_bins + self.lon_bins + self.sog_bins]) + self.lat_bins + self.lon_bins
          sog = (sog_bin - self.lat_bins - self.lon_bins) / self.sog_bins

          # Get cog (search in the fourth section of the vector)
          cog_bin = np.argmax(msg[self.lat_bins + self.lon_bins + self.sog_bins:]) + self.lat_bins + self.lon_bins + self.sog_bins
          cog = (cog_bin - self.lat_bins - self.lon_bins - self.sog_bins)/ self.cog_bins

          # Append the result to sparse_msgs
          sparse_msgs.append([lat, lon, sog, cog])

      # Convert the sparse_msgs list to a numpy array
      sparse_msgs = np.array(sparse_msgs)

      # If needed, scale back the values using the divm (scaling factor)
      # sparse_msgs *= divm

      return sparse_msgs

    def __len__(self):
        return len(self.dataTrackdFourHot)  #

    def __getitem__(self, idx):
        if idx >= len(self):
          idx=-1


        inputs =self.dataTrackdFourHot[idx]#-self.mean
        # return inputs

        # Normalize inputs by subtracting mean


        return torch.FloatTensor(inputs)
def create_dataloader(tracks, batch_size, lat_bins, lon_bins, sog_bins, cog_bins, shuffle=True):
    dataset = AISDataset(tracks, lat_bins, lon_bins, sog_bins, cog_bins)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return dataloader







In [ ]:
# @title create  dataset

batch_size=32  # @param {type: "number"}
lat_bins=LAT_BINS
lon_bins=LON_BINS
sog_bins=SOG_BINS


mean=[]
dataset = AISDataset(data, lat_bins, lon_bins, sog_bins, COG_BINS,mean)
print(f"Size of dataset: {len(dataset)}")
print(f"Total bins: {dataset.total_bins}")
print(f"num of message: {len(dataset.dataFourHot)}")

finsh get_four_hot_all_data 361
finsh processdata
Size of dataset: 361
Total bins: 260
num of message: 21351


# Models

In [ ]:
# @title VRNNAnomalyQuality

import torch
from torch import nn
from torch.distributions import Normal, Distribution
import math
from torch import Tensor
# Define Reparameterized Diagonal Gaussian Distribution
class ReparameterizedDiagonalGaussian(Distribution):
    def __init__(self, mu, log_sigma):
        assert mu.shape == log_sigma.shape, "Mu and log_sigma shapes must match."
        self.mu = mu
        self.sigma = log_sigma.exp()

    def sample_epsilon(self):
        return torch.empty_like(self.mu).normal_()

    def rsample(self):
        return self.mu + self.sigma * self.sample_epsilon()

    def log_prob(self, z):
        dist = Normal(self.mu, self.sigma)
        return dist.log_prob(z)


def kl_divergence_diag_gaussians(p: ReparameterizedDiagonalGaussian, q: ReparameterizedDiagonalGaussian) -> Tensor:
    log_var_p = p.sigma.log() * 2
    log_var_q = q.sigma.log() * 2

    kl_div = 0.5 * (
        (log_var_q - log_var_p).sum(dim=-1)
        + ((p.sigma ** 2 + (p.mu - q.mu) ** 2) / (q.sigma ** 2)).sum(dim=-1)
        - p.mu.size(-1)
    )
    return kl_div

# Define Encoder, Prior, and Decoder classes as before
class Encoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(Encoder, self).__init__()
        self.phi_x = nn.Sequential(nn.Linear(input_dim, latent_dim), nn.ReLU())
        self.encoder_net = nn.Linear(latent_dim * 2, 2 * latent_dim)

    def forward(self, x_enc, h):
        x_enc = self.phi_x(x_enc)
        enc = self.encoder_net(torch.cat([x_enc, h[0]], dim=-1))
        mu, log_sigma = torch.chunk(enc, 2, dim=-1)
        return ReparameterizedDiagonalGaussian(mu, log_sigma), x_enc


class Prior(nn.Module):
    def __init__(self, latent_dim):
        super(Prior, self).__init__()
        self.prior_net = nn.Linear(latent_dim, 2 * latent_dim)

    def forward(self, h):
        hidden = self.prior_net(h[0])
        mu, log_sigma = torch.chunk(hidden, 2, dim=-1)
        return ReparameterizedDiagonalGaussian(mu, log_sigma)


class Decoder(nn.Module):
    def __init__(self, latent_dim, output_dim):
        super(Decoder, self).__init__()
        self.phi_z = nn.Sequential(nn.Linear(latent_dim, latent_dim), nn.ReLU())
        self.decoder_net = nn.Linear(latent_dim * 2, output_dim)

    def forward(self, z, h):
        z_enc = self.phi_z(z)
        dec_input = torch.cat([z_enc, h[0]], dim=-1)
        logits = self.decoder_net(dec_input)
        return torch.sigmoid(logits), z_enc


# Define GRUState and LSTMState classes
class GRUState(nn.Module):
    def __init__(self, latent_dim):
        super(GRUState, self).__init__()
        self.gru = nn.GRU(latent_dim * 2, latent_dim, batch_first=True)

    def forward(self, x_enc, z_enc, h):
        gru_input = torch.cat([x_enc, z_enc], dim=-1).unsqueeze(1)
        _, h_next = self.gru(gru_input, h.unsqueeze(0))
        return h_next.squeeze(0), None  # No cell state needed


class LSTMState(nn.Module):
    def __init__(self, latent_dim):
        super(LSTMState, self).__init__()
        self.lstm = nn.LSTMCell(latent_dim * 2, latent_dim)

    def forward(self, x_enc, z_enc, h, c):
        lstm_input = torch.cat([x_enc, z_enc], dim=-1)
        h_next, c_next = self.lstm(lstm_input, (h, c))
        return h_next, c_next


# Define main VRNN model with a flexible state update mechanism
class VRNNAnomalyQuality_T(nn.Module):
    def __init__(self, input_dim, latent_dim, state_type="LSTM"):
        super(VRNNAnomalyQuality, self).__init__()

        self.encoder = Encoder(input_dim, latent_dim)
        self.prior = Prior(latent_dim)
        self.decoder = Decoder(latent_dim, input_dim)

        # Choose the state mechanism
        if state_type == "GRU":
            self.state_update = GRUState(latent_dim)
        elif state_type == "LSTM":
            self.state_update = LSTMState(latent_dim)
        else:
            raise ValueError("state_type must be either 'GRU' or 'LSTM'")

        self.state_type = state_type
        self.h_0 = torch.zeros(1, latent_dim)
        self.c_0 = torch.zeros(1, latent_dim) if state_type == "LSTM" else None
        self.threshold = 0.0001

    def forward(self, x, beta=1.0):
        batch_size, seq_len, _ = x.size()
        h = self.h_0.expand(batch_size, -1).to(x.device)
        c = self.c_0.expand(batch_size, -1).to(x.device) if self.c_0 is not None else None
        listlogis=[]

        total_loss, kl_divergence, recon_loss = 0, 0, 0

        for t in range(seq_len):
            x_t = x[:, t, :]

            prior_dist = self.prior((h, c))
            posterior_dist, x_enc = self.encoder(x_t, (h, c))

            z_t = posterior_dist.rsample()
            x_recon, z_enc = self.decoder(z_t, (h, c))


            # Update state based on chosen state mechanism
            if self.state_type == "LSTM":
                h, c = self.state_update(x_enc, z_enc, h, c)
            else:
                h, _ = self.state_update(x_enc, z_enc, h)



            kl_div = kl_divergence_diag_gaussians(posterior_dist, prior_dist).sum(dim=-1)
            kl_divergence += kl_div.mean()
            x_recon, z_enc = self.decoder(z_t, (h, c))
            recon_loss += nn.functional.binary_cross_entropy(x_recon, x_t, reduction='sum')
            listlogis.append(x_recon)
            # total_loss = recon_loss -beta * kl_divergence

        recon_loss /= seq_len
        kl_divergence /= seq_len
        total_loss = recon_loss + beta * kl_divergence
        return total_loss, recon_loss, kl_divergence
    def get_logis(self, x):
        batch_size, seq_len, _ = x.size()
        h = self.h_0.expand(batch_size, -1).to(x.device)
        c = self.c_0.expand(batch_size, -1).to(x.device) if self.c_0 is not None else None
        listlogis=[]

        total_loss, kl_divergence, recon_loss = 0, 0, 0
        for t in range(seq_len):
            x_t = x[:, t, :]

            prior_dist = self.prior((h, c))
            posterior_dist, x_enc = self.encoder(x_t, (h, c))
            z_t = posterior_dist.rsample()
            x_recon, z_enc = self.decoder(z_t, (h, c))
            listlogis.append(x_recon)
            if self.state_type == "LSTM":
                h, c = self.state_update(x_enc, z_enc, h, c)
            else:
                h, _ = self.state_update(x_enc, z_enc, h)

        x_rect=torch.stack(listlogis,dim=0)
        x_rect=x_rect.permute(1, 0, 2)
        return x_rect
    def calculate_anomaly_rate(self, inputs):

        batch_size = inputs.size(0)
        h = self.h_0.expand(batch_size, -1).contiguous().to(inputs.device)

        if self.state_type == 'LSTM':
            c = self.c_0.expand(batch_size, -1).contiguous().to(inputs.device)

        total_anomalies = 0
        total_points = 0

        for t in range(inputs.size(1)):
            x = inputs[:, t, :]

            posterior_dist, x_enc = self.encoder(x, (h, c) if self.state_type == 'LSTM' else (h,None))
            z = posterior_dist.rsample()

            x_recon, z_enc = self.decoder(z, (h, c) if self.state_type == 'LSTM' else(h,None))

            diff = torch.abs(x - x_recon)
            anomalies = (diff > self.threshold).sum(dim=-1)  #

            total_anomalies += anomalies.sum().item()
            total_points += x.numel()

            if self.state_type == 'LSTM':
                h, c = self.state_update(x_enc, z_enc, h, c)
            else:
                h ,_= self.state_update(x_enc, z_enc, h)

        # حساب نسبة الشذوذ
        anomaly_rate = total_anomalies / total_points
        return anomaly_rate
    def calc_mi(self, inputs):
        """
        Calculate mutual information (MI) for the VRNN model with support for both GRU and LSTM states.
        """
        batch_size = inputs.size(0)
        h = self.h_0.expand(batch_size, -1).contiguous().to(inputs.device)

        if self.state_type == 'LSTM':
            c = self.c_0.expand(batch_size, -1).contiguous().to(inputs.device)

        neg_entropy = 0
        log_qz = 0

        for t in range(inputs.size(1)):
            x = inputs[:, t, :]

            # Posterior and prior distributions
            posterior_dist, x_enc = self.encoder(x, (h, c) if self.state_type == 'LSTM' else (h,None))
            pz = self.prior((h, c) if self.state_type == 'LSTM' else h)

            # Sample from posterior
            mu = posterior_dist.mu
            logsigma = torch.log(posterior_dist.sigma)
            z = posterior_dist.rsample()
            _, z_enc = self.decoder(z, (h, c) if self.state_type == 'LSTM' else (h,None))

            # Update hidden state based on GRU or LSTM
            rnn_input = torch.cat([x_enc, z_enc], dim=1)
            if self.state_type == 'LSTM':
                h, c = self.state_update(x_enc, z_enc, h, c)
            else:
                h,_ = self.state_update(x_enc, z_enc, h)

            # Calculate mutual information terms
            neg_entropy += (-0.5 * self.encoder.encoder_net.out_features // 2 * math.log(2 * math.pi)
                            - 0.5 * (1 + 2 * logsigma).sum(-1)).mean()

            # Calculate log density for log_qz
            var = logsigma.exp() ** 2
            z = z.unsqueeze(1)
            mu = mu.unsqueeze(0)
            logsigma = logsigma.unsqueeze(0)
            dev = z - mu
            log_density = -0.5 * (dev ** 2 / var).sum(dim=-1) - 0.5 * (
                self.encoder.encoder_net.out_features // 2 * math.log(2 * math.pi) + (2 * logsigma).sum(dim=-1))
            log_qz1 = torch.logsumexp(log_density, dim=1) - math.log(batch_size)
            log_qz += log_qz1.mean(-1)

        # Calculate final MI value
        mi = (neg_entropy / inputs.size(1)) - (log_qz / inputs.size(1))
        return mi


In [ ]:
import torch
import torch.nn as nn
import math

class HybridStateUpdate(nn.Module):
    def __init__(self, latent_dim):
        super(HybridStateUpdate, self).__init__()
        self.gru = GRUState(latent_dim)
        self.lstm = LSTMState(latent_dim)
        self.alpha = nn.Parameter(torch.tensor(1.0))  # وزن المزيج بين GRU و LSTM

    def forward(self, x_enc, z_enc, h, c=None):
        h_gru, _ = self.gru(x_enc, z_enc, h)
        h_lstm, c_new = self.lstm(x_enc, z_enc, h, c)
        h_new = self.alpha * h_gru + (self.alpha) * h_lstm

        if c is not None:
            return h_new, c_new
        else:
            return h_new, None

class VRNNAnomalyQuality(nn.Module):
    def __init__(self, input_dim, latent_dim, state_type="Hybrid"):
        super(VRNNAnomalyQuality, self).__init__()

        self.encoder = Encoder(input_dim, latent_dim)
        self.prior = Prior(latent_dim)
        self.decoder = Decoder(latent_dim, input_dim)

        # Choose the state mechanism
        if state_type == "GRU":
            self.state_update = GRUState(latent_dim)
        elif state_type == "LSTM":
            self.state_update = LSTMState(latent_dim)
        elif state_type == "Hybrid":
            self.state_update = HybridStateUpdate(latent_dim)
        else:
            raise ValueError("state_type must be 'GRU', 'LSTM', or 'Hybrid'")

        self.state_type = state_type
        self.h_0 = torch.zeros(1, latent_dim)
        self.c_0 = torch.zeros(1, latent_dim) if state_type in ["LSTM", "Hybrid"] else None
        self.threshold = 0.0001

    def forward(self, x, beta=1.0):
        batch_size, seq_len, _ = x.size()
        h = self.h_0.expand(batch_size, -1).to(x.device)
        c = self.c_0.expand(batch_size, -1).to(x.device) if self.c_0 is not None else None
        listlogis = []

        total_loss, kl_divergence, recon_loss = 0, 0, 0

        for t in range(seq_len):
            x_t = x[:, t, :]

            prior_dist = self.prior((h, c) if self.c_0 is not None else (h, None))
            posterior_dist, x_enc = self.encoder(x_t, (h, c) if self.c_0 is not None else (h, None))

            z_t = posterior_dist.rsample()
            x_recon, z_enc = self.decoder(z_t, (h, c) if self.c_0 is not None else (h, None))

            if self.state_type == "Hybrid":
                h, c = self.state_update(x_enc, z_enc, h, c)
            elif self.state_type == "LSTM":
                h, c = self.state_update(x_enc, z_enc, h, c)
            else:
                h, _ = self.state_update(x_enc, z_enc, h)

            kl_div = kl_divergence_diag_gaussians(posterior_dist, prior_dist).sum(dim=-1)
            kl_divergence += kl_div.mean()
            recon_loss += nn.functional.binary_cross_entropy(x_recon, x_t, reduction='sum')
            listlogis.append(x_recon)

        recon_loss /= seq_len*10
        kl_divergence /= seq_len*10
        total_loss = recon_loss + beta * kl_divergence
        return total_loss, recon_loss, kl_divergence

    def get_logis(self, x):
        batch_size, seq_len, _ = x.size()
        h = self.h_0.expand(batch_size, -1).to(x.device)
        c = self.c_0.expand(batch_size, -1).to(x.device) if self.c_0 is not None else None
        listlogis = []

        for t in range(seq_len):
            x_t = x[:, t, :]

            prior_dist = self.prior((h, c) if self.c_0 is not None else (h, None))
            posterior_dist, x_enc = self.encoder(x_t, (h, c) if self.c_0 is not None else (h, None))
            z_t = posterior_dist.rsample()
            x_recon, z_enc = self.decoder(z_t, (h, c) if self.c_0 is not None else (h, None))
            listlogis.append(x_recon)

            if self.state_type == "Hybrid":
                h, c = self.state_update(x_enc, z_enc, h, c)
            elif self.state_type == "LSTM":
                h, c = self.state_update(x_enc, z_enc, h, c)
            else:
                h, _ = self.state_update(x_enc, z_enc, h)

        x_rect = torch.stack(listlogis, dim=0)
        x_rect = x_rect.permute(1, 0, 2)
        return x_rect

    def calculate_anomaly_rate(self, inputs):
        batch_size = inputs.size(0)
        h = self.h_0.expand(batch_size, -1).contiguous().to(inputs.device)
        c = self.c_0.expand(batch_size, -1).contiguous().to(inputs.device) if self.c_0 is not None else None

        total_anomalies = 0
        total_points = 0

        for t in range(inputs.size(1)):
            x = inputs[:, t, :]
            posterior_dist, x_enc = self.encoder(x, (h, c) if self.c_0 is not None else (h, None))
            z = posterior_dist.rsample()
            x_recon, z_enc = self.decoder(z, (h, c) if self.c_0 is not None else (h, None))

            diff = torch.abs(x - x_recon)
            anomalies = (diff > self.threshold).sum(dim=-1)

            total_anomalies += anomalies.sum().item()
            total_points += x.numel()

            if self.state_type == "Hybrid":
                h, c = self.state_update(x_enc, z_enc, h, c)
            elif self.state_type == "LSTM":
                h, c = self.state_update(x_enc, z_enc, h, c)
            else:
                h, _ = self.state_update(x_enc, z_enc, h)

        anomaly_rate = total_anomalies / total_points
        return anomaly_rate

    def calc_mi(self, inputs):
        batch_size = inputs.size(0)
        h = self.h_0.expand(batch_size, -1).contiguous().to(inputs.device)
        c = self.c_0.expand(batch_size, -1).contiguous().to(inputs.device) if self.c_0 is not None else None

        neg_entropy = 0
        log_qz = 0

        for t in range(inputs.size(1)):
            x = inputs[:, t, :]
            posterior_dist, x_enc = self.encoder(x, (h, c) if self.c_0 is not None else (h, None))
            pz = self.prior((h, c) if self.c_0 is not None else h)

            mu = posterior_dist.mu
            logsigma = torch.log(posterior_dist.sigma)
            z = posterior_dist.rsample()
            _, z_enc = self.decoder(z, (h, c) if self.c_0 is not None else (h, None))

            if self.state_type == "Hybrid":
                h, c = self.state_update(x_enc, z_enc, h, c)
            elif self.state_type == "LSTM":
                h, c = self.state_update(x_enc, z_enc, h, c)
            else:
                h, _ = self.state_update(x_enc, z_enc, h)

            neg_entropy += (-0.5 * self.encoder.encoder_net.out_features // 2 * math.log(2 * math.pi)
                            - 0.5 * (1 + 2 * logsigma).sum(-1)).mean()

            var = logsigma.exp() ** 2
            z = z.unsqueeze(1)
            mu = mu.unsqueeze(0)
            logsigma = logsigma.unsqueeze(0)
            dev = z - mu
            log_density = -0.5 * (dev ** 2 / var).sum(dim=-1) - 0.5 * (
                self.encoder.encoder_net.out_features // 2 * math.log(2 * math.pi) + (2 * logsigma).sum(dim=-1))
            log_qz1 = torch.logsumexp(log_density, dim=1) - math.log(batch_size)
            log_qz += log_qz1.mean(-1)

        mi = (neg_entropy / inputs.size(1)) - (log_qz / inputs.size(1))
        return mi


In [ ]:
# @title VRNNVotingModel

class VRNNVotingModel(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(VRNNVotingModel, self).__init__()
        self.vrnn_gru = VRNNAnomalyQuality(input_dim, latent_dim, state_type="GRU")
        self.vrnn_lstm = VRNNAnomalyQuality(input_dim, latent_dim, state_type="LSTM")
        self.state_type = "Voting"

    def forward(self, x, beta=1.0):
        # Train both models and get their losses
        loss_gru, recon_loss_gru, kl_div_gru = self.vrnn_gru(x, beta)
        loss_lstm, recon_loss_lstm, kl_div_lstm = self.vrnn_lstm(x, beta)

        # Take the average of the losses as the final loss (voting mechanism)
        total_loss = (loss_gru + loss_lstm) / 2
        recon_loss = (recon_loss_gru + recon_loss_lstm) / 2
        kl_divergence = (kl_div_gru + kl_div_lstm) / 2

        return total_loss, recon_loss, kl_divergence

    def calculate_anomaly_rate(self, inputs):
        # Calculate anomaly rate for both GRU and LSTM models
        anomaly_rate_gru = self.vrnn_gru.calculate_anomaly_rate(inputs)
        anomaly_rate_lstm = self.vrnn_lstm.calculate_anomaly_rate(inputs)

        # Take the average anomaly rate (voting mechanism)
        anomaly_rate = (anomaly_rate_gru + anomaly_rate_lstm) / 2
        return anomaly_rate

    def calc_mi(self, inputs):
        # Calculate mutual information for both GRU and LSTM models
        mi_gru = self.vrnn_gru.calc_mi(inputs)
        mi_lstm = self.vrnn_lstm.calc_mi(inputs)

        # Take the average mutual information (voting mechanism)
        mi = (mi_gru + mi_lstm) / 2
        return mi

    def get_logis(self, x):

       logis_gru=self.vrnn_gru.get_logis(x)
       logis_lstm=self.vrnn_lstm.get_logis(x)
       logis=(logis_gru+logis_lstm)/2
       return logis


In [ ]:
# @title Batch Dataset

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import random_split
def padded_sequences_batch(batch):

    sequences = []

    for sequence in batch:  # Iterate over each item in the batch
        sequences.append(sequence)

    # Pad the sequences
    padded_sequences = pad_sequence(sequences, batch_first=True, padding_value=0.0)
    padded_sequences = padded_sequences.float()


    return padded_sequences



batch_size = 32# @param {type: "number"}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_size=0.7 #@param {type: "number"}
train_size = int(train_size * len(dataset))
val_size= 0.15 #@param {type: "number"}
val_size = int(val_size * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True,collate_fn=padded_sequences_batch)
val_loader = DataLoader(val_dataset, batch_size=batch_size,collate_fn=padded_sequences_batch)
test_loader = DataLoader(test_dataset, batch_size=batch_size,collate_fn=padded_sequences_batch)


In [ ]:
len(train_loader)

12

In [ ]:
import wandb
# @title login  wandb

key='782b6a6e82bbb5a5348de0d3c7d40d1e76351e79' # @param {type: "string"}
wandb.login(key=key)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: modelasg. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# @title  train_model_wandb_long

import wandb
# تدريب مدى طويل  ملاحظة :
def train_model_wandb_long(model,optimizer ,train_loader,test_loader, epochs=10,step_eval=10,name_project="anomaly-detection"):

    lst_lossall=[]
    lst_klall=[]
    lst_reconall=[]
    lst_anomaly_rate=[]
    lst_test_anomaly_rate=[]
    lst_test_mi=[]
    # model.train()
    for epoch in range(epochs):
        epoch_loss, epoch_kl, epoch_recon = 0, 0, 0
        anomaly_rate = 0

        model.train()

        for i, x_batch in enumerate(train_loader):
            optimizer.zero_grad()
            batch_data = x_batch.to(device)
            batch_data = torch.clamp(batch_data , 0.0,0.99)
            total_loss, recon_loss, kl_divergence = model(batch_data)

            anomaly_rate += model.calculate_anomaly_rate(batch_data)

            total_loss.backward()
            optimizer.step()

            epoch_loss += total_loss.item()
            epoch_recon += recon_loss.item()
            epoch_kl += kl_divergence.item()

        wandb.log({
            "epoch": epoch + 1,
            "total_loss": epoch_loss / len(train_loader),
            "reconstruction_loss": epoch_recon / len(train_loader),
            "kl_divergence": epoch_kl / len(train_loader),
            "anomaly_rate": anomaly_rate,
        })
        lst_lossall.append(epoch_loss/len(train_loader))
        lst_klall.append(epoch_kl/len(train_loader))
        lst_reconall.append(epoch_recon/len(train_loader))
        lst_anomaly_rate.append(anomaly_rate/len(train_loader))

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss/len(train_loader):.4f}, Recon Loss: {epoch_recon/len(train_loader):.4f}, KL: {epoch_kl/len(train_loader):.4f}, Anomaly Rate: {anomaly_rate/len(train_loader):.4f}")
        if (epoch+1) % step_eval == 0:
            avg_anomaly_rate,mu_all, actual_trajectories, generated_trajectories = test_model_wandb(model, test_loader)

            log_trajectories_to_wandb(actual_trajectories, generated_trajectories,num_samples=2)
            lst_test_anomaly_rate.append(avg_anomaly_rate)
            lst_test_mi.append(np.mean(mu_all))


    return model,{f"{model.state_type}":{"Loss":lst_lossall,
                                                 "Recon-Loss":lst_reconall,
                                                 "KL-Loss":lst_klall,
                                                 "anomaly_rate":lst_anomaly_rate,
                                                 "avg_anomaly_rate":lst_test_anomaly_rate,
                                                 "avg_mi":lst_test_mi,
                                                 }}
def test_model_wandb(model, test_loader):
    model.eval()
    anomaly_rates = []
    mu_all=[]
    actual_trajectories, generated_trajectories = [], []

    with torch.no_grad():
        for i, x_batch in enumerate(test_loader):
            batch_data =dataset[350+i].unsqueeze(0).to(device)
            batch_data = torch.clamp(batch_data , 0.0, 0.99)



            anomaly_rate = model.calculate_anomaly_rate(batch_data)
            anomaly_rates.append(anomaly_rate)
            mu_all.append(model.calc_mi(batch_data).cpu().numpy())

            actual_trajectories.append(batch_data.cpu().numpy())
            x_rect=model.get_logis(batch_data)


            x_rect=x_rect.cpu().numpy()
            generated_trajectories.append(x_rect)

    avg_anomaly_rate = np.mean(anomaly_rates)
    wandb.log({"average_anomaly_rate_test": avg_anomaly_rate})
    print(f"Average Anomaly Rate (Test): {avg_anomaly_rate:.4f}")
    wandb.log({"average_mi_test": np.mean(mu_all)})
    print(f"Average MI (Test): {np.mean(mu_all):.4f}")
    return avg_anomaly_rate,np.mean(mu_all), actual_trajectories, generated_trajectories

# def log_trajectories_to_wandb(actual, generated, num_samples=5):


#     for i in range(num_samples):
#         data = {
#             f"Actual Trajectory {i+1}": actual[i][0,:,0:2],
#             f"Generated Trajectory {i+1}": generated[i][0,:,0:2],
#         }
#         wandb.log(data)
def log_trajectories_to_wandb(actual, generated, num_samples=5):
    num_samples = min(num_samples, len(actual))
    for i in range(num_samples):
        # Prepare data for line plot

        actual_traj = actual[i][0]  # Extract X and Y for actual trajectory
        generated_traj = generated[i][0]  # Extract X and Y for generated trajectory
        actual_traj=dataset.dense_to_sparse(actual_traj)
        generated_traj=dataset.dense_to_sparse(generated_traj)
        actual_traj=actual_traj[:,0:2]
        generated_traj=generated_traj[:,0:2]


        # Convert to format suitable for W&B
        table = wandb.Table(data=[
            [int(x*LAT_BINS), int(y*LON_BINS), "Actual"] for x, y in actual_traj
        ] + [
            [int(x*LAT_BINS), int(y*LON_BINS), "Generated"] for x, y in generated_traj
        ], columns=["x", "y", "Type"])

        # Log the line plot
        wandb.log({f"Trajectory {i+1}": wandb.plot.line(
            table, "x", "y", "Type", title=f"Trajectory {i+1}",



        )})





In [ ]:
# @title train_model_sciles

import wandb

# تدريب  شرائح
def train_model_sciles(model,optimizer ,train_loader,test_loader, epochs=10,step_eval=10,name_project="anomaly-detection"):

    # model.train()
    size_chack=10
    lst_lossall=[]
    lst_klall=[]
    lst_reconall=[]
    lst_anomaly_rate=[]
    lst_test_anomaly_rate=[]
    lst_test_mi=[]


    for epoch in range(epochs):
        epoch_loss, epoch_kl, epoch_recon = 0, 0, 0
        anomaly_rate = 0
        model.train()


        for i, x_batch in enumerate(train_loader):
            optimizer.zero_grad()
            batch_data = x_batch.to(device)
            batch_data = torch.clamp(batch_data , 0.0, 0.99)

            chacks=[ chack for chack in range(size_chack)]
            sub_loss,sub_recon_loss, sub_kl_divergence = 0, 0, 0
            for chack in chacks:
                sub_batch=batch_data[:,chack:chack+size_chack,:]

                optimizer.zero_grad()

                total_loss, recon_loss, kl_divergence = model(sub_batch)




                total_loss.backward()


                optimizer.step()

                sub_loss += total_loss.item()
                sub_recon_loss += recon_loss.item()
                sub_kl_divergence += kl_divergence.item()


            epoch_loss += sub_loss/len(chacks)
            epoch_recon += sub_recon_loss/len(chacks)
            epoch_kl += sub_kl_divergence/len(chacks)
            anomaly_rate += model.calculate_anomaly_rate(batch_data)


        wandb.log({
            "epoch": epoch + 1,
            "total_loss": epoch_loss / len(train_loader),
            "reconstruction_loss": epoch_recon / len(train_loader),
            "kl_divergence": epoch_kl / len(train_loader),
            "anomaly_rate": anomaly_rate,
        })
        lst_lossall.append(epoch_loss/len(train_loader))
        lst_klall.append(epoch_kl/len(train_loader))
        lst_reconall.append(epoch_recon/len(train_loader))
        lst_anomaly_rate.append(anomaly_rate/len(train_loader))

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss/len(train_loader):.4f}, Recon Loss: {epoch_recon/len(train_loader):.4f}, KL: {epoch_kl/len(train_loader):.4f}, Anomaly Rate: {anomaly_rate/len(train_loader):.4f}")
        if (epoch+1) % step_eval == 0:
            avg_anomaly_rate,mu_all, actual_trajectories, generated_trajectories = test_model_wandb(model, test_loader)

            log_trajectories_to_wandb(actual_trajectories, generated_trajectories,num_samples=2)
            lst_test_anomaly_rate.append(avg_anomaly_rate)
            lst_test_mi.append(np.mean(mu_all))


    return model,{f"{model.state_type}":{"Loss":lst_lossall,
                                                 "Recon-Loss":lst_reconall,
                                                 "KL-Loss":lst_klall,
                                                 "anomaly_rate":lst_anomaly_rate,
                                                 "avg_anomaly_rate":lst_test_anomaly_rate,
                                                 "avg_mi":lst_test_mi,
                                                 }}
def test_model_wandb(model, test_loader):
    model.eval()
    anomaly_rates = []
    mu_all=[]
    actual_trajectories, generated_trajectories = [], []

    with torch.no_grad():
        for i, x_batch in enumerate(test_loader):
            batch_data =dataset[350+i].unsqueeze(0).to(device)
            batch_data = torch.clamp(batch_data , 0.0, 0.99)



            anomaly_rate = model.calculate_anomaly_rate(batch_data)
            anomaly_rates.append(anomaly_rate)
            mu_all.append(model.calc_mi(batch_data).cpu().numpy())

            actual_trajectories.append(batch_data.cpu().numpy())
            x_rect=model.get_logis(batch_data)


            x_rect=x_rect.cpu().numpy()
            generated_trajectories.append(x_rect)

    avg_anomaly_rate = np.mean(anomaly_rates)
    wandb.log({"average_anomaly_rate_test": avg_anomaly_rate})
    print(f"Average Anomaly Rate (Test): {avg_anomaly_rate:.4f}")
    wandb.log({"average_mi_test": np.mean(mu_all)})
    print(f"Average MI (Test): {np.mean(mu_all):.4f}")
    return avg_anomaly_rate,np.mean(mu_all), actual_trajectories, generated_trajectories

# def log_trajectories_to_wandb(actual, generated, num_samples=5):


#     for i in range(num_samples):
#         data = {
#             f"Actual Trajectory Four-hot  {i+1}": actual[i][0,:,0:2],
#             f"Generated Trajectory Four-hot {i+1}": generated[i][0,:,0:2],
#         }
#         wandb.log(data)
def log_trajectories_to_wandb(actual, generated, num_samples=5):

    num_samples = min(num_samples, len(actual))
    for i in range(num_samples):
        actual_traj = actual[i][0]
        generated_traj = generated[i][0]
        actual_traj=dataset.dense_to_sparse(actual_traj)
        generated_traj=dataset.dense_to_sparse(generated_traj)
        actual_traj=actual_traj[:,0:2]
        generated_traj=generated_traj[:,0:2]
        table = wandb.Table(data=[
            [int(x*LAT_BINS), int(y*LON_BINS), "Actual"] for x, y in actual_traj
        ] + [
            [int(x*LAT_BINS), int(y*LON_BINS), "Generated"] for x, y in generated_traj
        ], columns=["x", "y", "Type"])

        wandb.log({f"Trajectory {i+1}": wandb.plot.line(
            table, "x", "y", "Type", title=f"Trajectory {i+1}",



        )})





In [ ]:
# @title create model  and  optimizer

input_dim=dataset.total_bins
latent_dim=100 # @param {type: "number"}
type_model='Hybrid' # @param ['Hybrid','GRU','LSTM','GRU_LSTM']
# Initialize model
if type_model=='GRU_LSTM':
    model = VRNNVotingModel(input_dim=input_dim, latent_dim=latent_dim).to(device)
else :
    model = VRNNAnomalyQuality(input_dim=input_dim, latent_dim=latent_dim,state_type=type_model).to(device)

# @markdown  optimizer
lr=0.001 # @param {type: "number"}
beta0=0.8 #@param {type: "number"}
beta1=0.999 # @param {type: "number"}
eps=1e-08 # @param {type: "number"}
weight_decay=0.0 # @param {type: "number"}
optimizer = torch.optim.AdamW(model.parameters(), lr=lr,betas=(beta0,beta1), eps=1e-08, weight_decay=0)

print('created model')

created model


In [ ]:
def  create_model(input_dim,latent_dim,type_model):
    if type_model=='GRU_LSTM':
        model = VRNNVotingModel(input_dim=input_dim, latent_dim=latent_dim).to(device)
        return model
    else :
        model = VRNNAnomalyQuality(input_dim=input_dim, latent_dim=latent_dim,state_type=type_model).to(device)
        return model
    return model

In [ ]:
# @title Training And Testing

name_project='anomaly-detection-listm_200' # @param {type: "string"}
wandb.init(
    project=name_project,
    name="trajectory-model-listall",
    config={
        "epochs": 10,
        "batch_size": 32,
        "learning_rate": 0.001,
    },
    )
name_project_wandb='anomaly-detection-listm_200' # @param {type: "string"}
epochs=200 # @param {type: "number"}
step_eval=5 # @param {type: "number"}
type_train='Long' # @param ['Long','sciles']
if type_train=='Long':
    outs=train_model_wandb_long(model,optimizer ,train_loader,test_loader, epochs=epochs,step_eval=step_eval,name_project=name_project_wandb)
else :
    outs=train_model_sciles(model,optimizer ,train_loader,test_loader, epochs=epochs,step_eval=step_eval,name_project=name_project_wandb)

Epoch 1/200, Loss: 429.7744, Recon Loss: 406.3935, KL: 23.3809, Anomaly Rate: 1.0000
Epoch 2/200, Loss: 117.6129, Recon Loss: 84.4217, KL: 33.1912, Anomaly Rate: 1.0000
Epoch 3/200, Loss: 42.2382, Recon Loss: 32.3559, KL: 9.8824, Anomaly Rate: 1.0000
Epoch 4/200, Loss: 31.9924, Recon Loss: 26.9041, KL: 5.0883, Anomaly Rate: 1.0000
Epoch 5/200, Loss: 28.2342, Recon Loss: 24.8106, KL: 3.4235, Anomaly Rate: 0.9999
Average Anomaly Rate (Test): 0.9997
Average MI (Test): -0.2018
Epoch 6/200, Loss: 26.2126, Recon Loss: 23.6671, KL: 2.5456, Anomaly Rate: 0.9996
Epoch 7/200, Loss: 24.6336, Recon Loss: 22.6507, KL: 1.9828, Anomaly Rate: 0.9979
Epoch 8/200, Loss: 24.5474, Recon Loss: 22.8404, KL: 1.7071, Anomaly Rate: 0.9927
Epoch 9/200, Loss: 24.0772, Recon Loss: 22.6323, KL: 1.4449, Anomaly Rate: 0.9793
Epoch 10/200, Loss: 22.4255, Recon Loss: 21.1513, KL: 1.2742, Anomaly Rate: 0.9430
Average Anomaly Rate (Test): 0.9533
Average MI (Test): -0.0224
Epoch 11/200, Loss: 22.7365, Recon Loss: 21.2299

In [ ]:
# prompt: outs save and download

import os
import pickle

# Save the model and training history
model_filename = 'trained_model_long200.pth'
history_filename = 'trained_model_long200.pkl'

torch.save(outs[0].state_dict(), model_filename)
with open(history_filename, 'wb') as f:
    pickle.dump(outs[1], f)

# Download the saved files
from google.colab import files
files.download(model_filename)
files.download(history_filename)

In [ ]:
def  train_model_thrd(inputs):

     input_dim,latent_dim,type_model,train_loader,test_loader=inputs
     model=create_model(input_dim,latent_dim,type_model)

     name_project_wandb='anomaly-detection-listm_200' # @param {type: "string"}
     epochs=20 # @param {type: "number"}
     step_eval=5 # @param {type: "number"}
     # @markdown  optimizer
      # @markdown  optimizer
     lr=0.001 # @param {type: "number"}
     beta0=0.8 #@param {type: "number"}
     beta1=0.999 # @param {type: "number"}
     eps=1e-08 # @param {type: "number"}
     weight_decay=0.0 # @param {type: "number"}
     optimizer = torch.optim.AdamW(model.parameters(), lr=lr,betas=(beta0,beta1), eps=1e-08, weight_decay=0)
     type_train='sciles' # @param ['Long','sciles']
     if type_train=='Long':
        outs=train_model_wandb_long(model,optimizer ,train_loader,test_loader, epochs=epochs,step_eval=step_eval,name_project=name_project_wandb)
     else :
        outs=train_model_sciles(model,optimizer ,train_loader,test_loader, epochs=epochs,step_eval=step_eval,name_project=name_project_wandb)
     torch.save(outs[0].state_dict(), f'/content/drive/MyDrive/t/model_{type_model}.pth')
     return outs


In [ ]:
from concurrent.futures import ThreadPoolExecutor
#LSTM','GRU_LSTM'
typemodels=['Hybrid']
input_dim=dataset.total_bins
latent_dim=100 # @param {type: "number"}
max_=10
name_project='anomaly-detection-listm_200' # @param {type: "string"}
wandb.init(
    project=name_project,
    name="trajectory-model-listall",
    config={
        "epochs": 10,
        "batch_size": 32,
        "learning_rate": 0.001,
    },
    )
inputs=[]
for type_model in typemodels:
    inputs.append((input_dim,latent_dim,type_model,train_loader,test_loader))

with ThreadPoolExecutor(max_workers=16) as executor:
    results = list(executor.map(train_model_thrd, inputs))


In [ ]:
import torch
import torch.nn as nn

# Define the biLSTM model
class BiLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, bidirectional=True):
        super(BiLSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional

        # Define the biLSTM layer
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            bidirectional=bidirectional,
            batch_first=True
        )

        # Define a fully connected layer for output
        self.fc = nn.Linear(hidden_size * 2 if bidirectional else hidden_size, output_size)

    def forward(self, x):
        # Pass the input through the biLSTM layer
        lstm_out, _ = self.lstm(x)  # lstm_out shape: (batch_size, seq_length, hidden_size * 2 if bidirectional)

        # Use the final hidden states (forward and backward)
        if self.bidirectional:
            out = torch.cat((lstm_out[:, -1, :self.hidden_size], lstm_out[:, 0, self.hidden_size:]), dim=1)
        else:
            out = lstm_out[:, -1, :]

        # Pass through the fully connected layer
        out = self.fc(out)
        return out

# Example usage
if __name__ == "__main__":
    # Parameters
    input_size = 10   # Size of each input vector
    hidden_size = 20  # Number of features in the hidden state
    output_size = 2   # Number of output classes
    seq_length = 5    # Length of the input sequences
    batch_size = 3    # Number of sequences in a batch

    # Create an instance of the biLSTM model
    model = BiLSTMModel(input_size, hidden_size, output_size)

    # Create some random input data
    inputs = torch.randn(batch_size, seq_length, input_size)  # (batch_size, seq_length, input_size)

    # Forward pass
    outputs = model(inputs)

    # Print the output shape
    print("Output shape:", outputs.shape)  # Expected: (batch_size, output_size)


Output shape: torch.Size([3, 2])


In [ ]:
model,metrics=results
